In [4]:
from bs4 import BeautifulSoup
from requests import TooManyRedirects
import re
import requests
from datetime import datetime
import os
import pandas as pd
import regex as re
# ------- import ----------
from scrape_function import * 
from scrape_util import *

In [5]:
home_movie = concat_df_from_dir('movies_at_home')
home_movie['platform'] = home_movie['date_list'].apply(lambda x:x.split()[0])
home_movie['movie_date'] = home_movie['date_list'].apply(lambda x:' '.join(x.split()[1:]))
home_movie['movie_date'] = pd.to_datetime(home_movie['movie_date'], format = '%b %d, %Y')
home_movie = home_movie[['title','critic_sentiment','critic_score','audience_setiment','audience_score','platform','movie_date']]
home_movie = home_movie
str_date = datetime(2020, 1, 1)
home_movie = home_movie[home_movie['movie_date'] > str_date]
all_title = home_movie['title'].tolist()

In [6]:
def get_movie_info(movie_name):
    #movie_name = 'Hocus Pocus'
    movie_name = re.sub(r"[^a-zA-Z]{1,}", "_", movie_name).lower()
    if movie_name[-1] == '_':
        movie_name = movie_name[:-1]
    print(f'Try to get response from https://www.rottentomatoes.com/m/{movie_name}')
    response = requests.get(f'https://www.rottentomatoes.com/m/{movie_name}')
    soup = BeautifulSoup(response.content, 'html.parser')
    d = {}
    #<div class="category-wrap" data-qa="item">
    content_wrap = soup.find_all('div',class_ = 'category-wrap', attrs={"data-qa": "item"})
    for _ in content_wrap:
        k = ','.join([x.contents[0] for x in _.find_all(attrs={"data-qa": "item-label"})])
        v = [','.join([x.contents[0] for x in _.find_all(attrs={"data-qa": "item-value"})])]
        d[k] = v
    return pd.DataFrame(d)

In [7]:
first_iter = True
istr = 6000
istp = 6227
for title in all_title[istr:istp]:
    try:
        if first_iter:
            df = get_movie_info(title)
            df['title'] = title
            first_iter = False
        else:
            df_ = get_movie_info(title)
            df_['title'] = title
            df = df._append(df_, ignore_index = True) 
    except Exception as e:
        print(f'Failed to get review from {title}:  {e}')
df.to_csv(f'movie_info/movie_info_{istr}_{istp}.csv', index = False)

Try to get response from https://www.rottentomatoes.com/m/a_little_white_lie
Try to get response from https://www.rottentomatoes.com/m/boys_from_county_hell
Try to get response from https://www.rottentomatoes.com/m/game_of_love
Try to get response from https://www.rottentomatoes.com/m/mimi
Try to get response from https://www.rottentomatoes.com/m/one_day_as_a_lion
Try to get response from https://www.rottentomatoes.com/m/the_abandoned
Try to get response from https://www.rottentomatoes.com/m/the_independent
Try to get response from https://www.rottentomatoes.com/m/the_saint_of_second_chances
Try to get response from https://www.rottentomatoes.com/m/a_thousand_cuts
Try to get response from https://www.rottentomatoes.com/m/black_beauty
Try to get response from https://www.rottentomatoes.com/m/dangerous_waters
Try to get response from https://www.rottentomatoes.com/m/ham_on_rye
Try to get response from https://www.rottentomatoes.com/m/honeydew
Try to get response from https://www.rottento

In [9]:
df

,Director,Genre,Runtime,title,Screenwriter,Production Co,Original Language,Release Date (Streaming),Producer,Distributor,Rating,Release Date (Theaters),Rerelease Date (Theaters),Box Office (Gross USA),Sound Mix,Aspect Ratio,Release Date (DVD),Creator
0,John Njama,Drama,1h 40m,A Little White Lie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chris Baugh,"Comedy,Horror",1h 30m,Boys from County Hell,"Chris Baugh,Brendan Mullin","Blinder Films,Endeavor Content,Six Mile Hill P...",British English,"Apr 22, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bobby Roth,Romance,1h 40m,Game of Love,NaN,NaN,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Paul Stein,Romance,1h 38m,Mimi,NaN,NaN,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Swab,"Crime,Drama,Comedy",1h 27m,One Day as a Lion,Scott Caan,"Roxwell Films,Universal Studios,Three Point Ca...",English,"Apr 7, 2023",Jeremy M. Rosen,Lionsgate,R (Sexual References|Pervasive Language|Some V...,"Apr 4, 2023, Limited",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Jon Keeyes,"Action,Drama",1h 33m,Code Name Banshee,Matthew Rogers,"Highland Myst Films,Bee-Hive Productions,LB En...",English,"Jul 1, 2022","Jordan Yale Levine,Jordan Beckerman,Jon Keeyes...",Screen Media Films,NaN,"Jul 1, 2022, Limited",NaN,NaN,NaN,Scope (2.35:1),NaN,NaN
202,Chris Skotchdopole,"Comedy,Drama,Mystery & Thriller",1h 43m,Crumb Catcher,Chris Skotchdopole,"Glass Eye Pix,Gigantic Pictures",English,"Aug 20, 2024","Larry Fessenden,Brian Devine,Chadd Harbold,Bon...",Doppelgänger Releasing,NaN,"Jul 19, 2024, Limited",NaN,$39.0K,NaN,NaN,NaN,NaN
203,Katrina McPherson,Documentary,NaN,Force of Nature,NaN,NaN,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,Jamie Roberts,Documentary,1h 30m,Four Hours at the Capitol,NaN,NaN,English,"Oct 20, 2021",Dan Reed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
len(all_title)

6227